# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [34]:
#from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.text_utils import DocumentLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [35]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `DocumentLoader` class:

```python
def load_text_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [36]:
#text_loader = TexiFileLoader("data/PMarcaBlogs.txt")
text_loader = DocumentLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [37]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [38]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [39]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [40]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


=================================================================================================================================================================
##### ✅ Answer:

1. Yes, by using the optional `dimensions` parameter when creating the embedding.
2. OpenAI uses a technique called [Matryoshka Representation Learning](https://arxiv.org/abs/2205.13147), , which trains embeddings so that **truncated versions** (e.g., first 256 dimensions out of 1536) retain meaningful semantic information. This allows to use smaller dimensions for quick semantic filtering and larger dimensions for precise matching - all from the same embedding vector.

=================================================================================================================================================================

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [41]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

=================================================================================================================================================================
##### ✅ Answer:
Before listing the benefits of using an async approach to collect embeddings, it's important to understand the core difference between **sync** and **async**:

- **SYNC (Synchronous)**: The application executes tasks sequentially - one at a time. Each task must complete before the next one begins. If you need 100 embeddings and each API call takes 2 seconds, the total time would be 200 seconds (100 × 2).

- **ASYNC (Asynchronous)**: The application can initiate multiple tasks simultaneously without waiting for each to complete. The same 100 embedding requests can be started at once and complete in approximately 2 seconds (the time of a single API call).

###### Benefits of Async for Embedding Collection:

1. **Speed Improvement**: 100x faster processing (200 seconds → 2 seconds in the example above)
2. **Better Resource Utilization**: Maximizes network bandwidth and API throughput instead of leaving resources idle
3. **Scalability**: Enables processing of large datasets (thousands of documents) in reasonable timeframes
4. **Cost Efficiency**: Reduced processing time translates to lower computational costs

=================================================================================================================================================================

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [42]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

=================================================================================================================================================================

##### ✅ Answer:

Yes, there are some ways to achieve more reproducible outputs (same output repeated):
- **Seed** - Set the same integer value across requests, is a number that controls the randomness in AI decision-making.
- **Temperature** - It controls how creative or random the AI is, the lower the value the more deterministic (more consistent)

There are other parameters like **top_p values** which is similar to temperature but instead of using ALL possible words, it only considers the most likely words that add up to a certain probability.

=================================================================================================================================================================


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [43]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [44]:
print(response)

Hello! The best way to write a loop in Python depends on what you're trying to achieve. Generally, Python provides two main types of loops:

1. **for loop** — used for iterating over a sequence (like a list, tuple, string, or range).
2. **while loop** — repeats as long as a certain condition is true.

Here are some tips and examples for writing clean and efficient loops:

### Using a `for` loop (most common and Pythonic)
```python
# Loop through a list
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```

```python
# Loop a specific number of times using range()
for i in range(5):
    print(i)
```

### Using a `while` loop
```python
count = 0
while count < 5:
    print(count)
    count += 1
```

### Best practices:
- Use `for` loops when you know the sequence or number of iterations.
- Use `while` loops for conditional repetition.
- Avoid infinite loops unless intentional (make sure your condition will become False at some point).
- Use meaningful variable n

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [45]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [46]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [47]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder, who was previously successful and experienced in a particular function (e.g., product management, sales, marketing), hires a weak executive to run that same function after becoming CEO. This happens because the CEO has a hard time letting go of the function that brought them success and wants to remain "the man" in that area. The term is named after Michael Eisner, the former CEO of Disney who had been a brilliant TV network executive but, after acquiring ABC, the network fell to fourth place. Eisner's response was that if he had extra time, he could have turned ABC around himself, but he did not — illustrating the problem of not delegating strong executives for the former specialty area and instead possibly settling for weaker ones to maintain control. (Source 1)

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

=================================================================================================================================================================

##### ✅ Answer:

**Chain-of-Thought (CoT) Prompting** involves asking the LLM to "think step by step" or "explain your reasoning" before providing the final answer. This strategy encourages the model to break down complex problems, show intermediate reasoning steps, and provide more comprehensive responses.

**Example implementation:**
```
"Based on the provided context, think step by step about the Michael Eisner Memorial Weak Executive Problem:
1. First, identify what the problem is
2. Then, explain why it happens  
3. Finally, provide specific examples from the context
Please show your reasoning for each step."
```

**Other effective strategies:**
- **Few-Shot Prompting** - Provide 2-3 examples of the kind of detailed, thoughtful response you want. The model will pattern-match and produce similar quality outputs.
- **Role-Based Prompting** - Ask the model to take on the role of an expert: "As a senior data scientist with 10 years of experience, explain..."
- **Prompt Chaining** - Break complex tasks into sequential prompts where each builds on the previous output. Instead of "Write a business plan," use the following example which allows deeper focus on each component:
    1. "Identify key market factors,"
    2. "Analyze those factors for opportunities,"
    3. "Develop strategy based on that analysis."

=================================================================================================================================================================

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [52]:
# ENHANCEMENT: PDF Support
# ===================================================
import os

try:
    from aimakerspace.text_utils import DocumentLoader
    
    # Find and display all PDF files in data directory
    data_dir = "data"
    pdf_files = [f for f in os.listdir(data_dir) if f.endswith('.pdf')]
    
    print(f"Found {len(pdf_files)} PDF files:")
    for pdf_file in pdf_files:
        print(f"{pdf_file}")
    
    # Load ALL files in data directory (both .txt and .pdf)
    enhanced_loader = DocumentLoader(data_dir)
    documents = enhanced_loader.load_documents()
    
    print(f"\nLoaded {len(documents)} total documents")
    print(f"Supported types: {enhanced_loader.get_supported_extensions()}")
    
    # Build RAG system with ALL content
    enhanced_split_documents = text_splitter.split_texts(documents)
    enhanced_vector_db = VectorDatabase()
    enhanced_vector_db = asyncio.run(enhanced_vector_db.abuild_from_list(enhanced_split_documents))
    
    # Create the RAG pipeline (ready for questions!) - Make it GLOBAL
    global enhanced_rag_pipeline
    enhanced_rag_pipeline = RetrievalAugmentedQAPipeline(
        vector_db_retriever=enhanced_vector_db,
        llm=chat_openai,
        response_style="detailed",
        include_scores=True
    )
    
    # Define the easy question function
    def ask_question(question: str, k: int = 3, show_scores: bool = True):
        """
        Simple function to ask questions to your RAG system.
        
        Args:
            question (str): Your question
            k (int): Number of relevant sources to retrieve (default: 3)
            show_scores (bool): Whether to show similarity scores (default: True)
        
        Usage in new cells:
            ask_question("What is information security?")
        """
        print(f"Question: {question}")
        print("-" * 50)
        
        try:
            result = enhanced_rag_pipeline.run_pipeline(question, k=k)
            
            print(f"Answer: {result['response']}")
            print(f"\nFound {result['context_count']} relevant sources")
            
            if show_scores and result['similarity_scores']:
                print(f"Similarity scores: {result['similarity_scores']}")
                
        except Exception as e:
            print(f"Error: {e}")
            print("Make sure the enhanced RAG pipeline is properly set up.")
    
    print(f"Enhanced RAG system ready! Contains content from all {len(documents)} documents")
    print("Question function ready! Usage: ask_question('Your question here')")
    print("You can now use ask_question() in any new cell!")
    
except ImportError as e:
    print(f"Import Error: {e}")
    print("Solution: Go to Kernel → Restart Kernel, then re-run all cells")
    
except Exception as e:
    print(f"Error: {e}")
    print("Make sure you've run all previous cells first!")


Found 1 PDF files:
ISO 27001 - Information security, cybersecurity and privacy protection — Information security management systems.pdf

Loaded 2 total documents
Supported types: ['.txt', '.pdf']
Enhanced RAG system ready! Contains content from all 2 documents
Question function ready! Usage: ask_question('Your question here')
You can now use ask_question() in any new cell!


In [54]:
ask_question("What are the key principles of information security?")

Question: What are the key principles of information security?
--------------------------------------------------
Answer: Based on the provided context, the key principles of information security include:

1. Protection of Intellectual Property Rights (5.32): Organizations must implement appropriate procedures to protect intellectual property rights.

2. Protection of Records (5.33): Records must be protected from loss, destruction, falsification, unauthorized access, and unauthorized release.

3. Privacy and Protection of Personal Identifiable Information (PII) (5.34): Organizations should identify and meet requirements to preserve privacy and protect PII in accordance with applicable laws, regulations, and contractual requirements.

4. Independent Review of Information Security (5.35): An organization’s approach to managing information security—including people, processes, and technologies—shall be reviewed independently at planned intervals or when significant changes occur.

5. Com

In [55]:
ask_question("What is the Michael Eisner Memorial Weak Executive Problem?",
    k=5,                    # More sources
    show_scores=False       # Hide similarity scores
)

Question: What is the Michael Eisner Memorial Weak Executive Problem?
--------------------------------------------------
Answer: The "Michael Eisner Memorial Weak Executive Problem" refers to the phenomenon where a CEO or startup founder who used to excel in a particular function hires a weak executive to run that same function. This often happens because the CEO has difficulty letting go of the function that originally brought them success, and as a result, they intentionally or unintentionally hire someone weaker in that role so they can continue to be the dominant figure ("the man") in that area. For example, a CEO who was formerly a product manager might have a weak product management executive, or a CEO who was in sales might have a weak sales executive.

The problem is named after Michael Eisner, the CEO of Disney who was previously a brilliant TV network executive. When he bought ABC at Disney, ABC dropped to fourth place, and Eisner's response was that if he had two extra days 

In [56]:
ask_question("How do I ride a bike?")

Question: How do I ride a bike?
--------------------------------------------------
Answer: I don't know. The provided context does not contain any information on how to ride a bike.

Found 3 relevant sources
Similarity scores: ['Source 1: 0.183', 'Source 2: 0.182', 'Source 3: 0.173']


### 📚 Summary: Enhanced RAG with Super Easy Question System

**What you now have:**
- **Multi-format support**: Loads ALL `.txt` and `.pdf` files from your data directory  
- **One-line questioning**: Just use `ask_question("your question")` in any new cell
- **Comprehensive knowledge base**: Contains content from PMarca blogs AND ISO 27001 PDF

**Setup (one-time only):**
1. Run all cells up to the original RAG content (cells 1-47)
2. Run the **Enhancement cell** (contains PDF support + question function)
3. Done! Now you can ask questions anywhere

**How to add new questions:**
1. Create a new cell
2. Copy: `ask_question("Your question here")`
3. Change the question and run!

**No more complex setup needed** - just one simple function call per question! 🎉

**Pro tips:**
- Use `k=5` for more sources: `ask_question("question", k=5)`
- Hide scores: `ask_question("question", show_scores=False)`
- Copy the template above to get started quickly
